In [1]:
# Conventionally people rename the pandas import to pd for brevity
import pandas as pd

In [2]:
# Load in the data and preview it
sales = pd.read_csv('home_data.csv') 
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
# Order by zipcode
sales = sales.sort_values('zipcode')

# Create a list of the unique zipcodes (numpy.ndarray)
unique_zipcodes = sales['zipcode'].unique()

# Create list of dataframes by zipcode
list_of_df = []

for zipcode in unique_zipcodes:
   # Create df for each zipcode
   df = sales[sales['zipcode'] == zipcode]
   # Append to the list_of_df
   list_of_df.append(df)

In [4]:
from sklearn.model_selection import train_test_split

list_of_df_train = []
list_of_df_test = []

# Split each dataframe into train (80%) and test data (20%) 
for df in list_of_df:
   train_data, test_data = train_test_split(df, test_size=0.2)
   list_of_df_train.append(train_data)
   list_of_df_test.append(test_data)

print(f"# of zipcodes:", len(unique_zipcodes))
print(f"# of (train) dataframes:", len(list_of_df_train))
print(f"# of (test) dataframes:", len(list_of_df_test))


# of zipcodes: 70
# of (train) dataframes: 70
# of (test) dataframes: 70


In [31]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error 

# List features to use for model to predict 
basic_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']

list_of_models = []
list_of_rmse_basic_train = []
# Basic Model
for df in list_of_df_train:
   y = df.price # actual price column of training set
   X = df[basic_features] # rest of dataframe data
   # Create and train the model
   basic_model = linear_model.LinearRegression().fit(X, y)
   # Store the model into list of models
   list_of_models.append(basic_model)
   # Predict prices using the model
   y = df.price # actual price column of training set
   X = df[basic_features]
   y_pred = basic_model.predict(X)
   train_rmse_basic = mean_squared_error(y, y_pred, squared=False) #False = rmse
   list_of_rmse_basic_train.append(train_rmse_basic)

print(f"# of ML models: ", len(list_of_models))
print(f"# of Root Mean Squered Error: ", len(list_of_rmse_basic_train))


# of ML models:  70
# of Root Mean Squered Error:  70


In [33]:
# Comparing with Test Data
list_of_rmse_basic_test = []

i = 0
while i < len(list_of_models):
   df = list_of_df_test[i]
   y = df.price
   X = df[basic_features]
   y_pred = list_of_models[i].predict(X)
   test_rmse_basic = mean_squared_error(y, y_pred, squared=False) #False = rmse
   list_of_rmse_basic_test.append(test_rmse_basic)
   i += 1

df = pd.DataFrame({
   'unique_zipcodes': unique_zipcodes,
   'list_of_rmse_basic_train': list_of_rmse_basic_train,
   'list_of_rmse_basic_test': list_of_rmse_basic_test
   })


,unique_zipcodes,list_of_rmse_basic_train,list_of_rmse_basic_test
0,98001,48361.684334,66839.349040
1,98002,30146.258142,30965.579103
2,98003,55546.505189,61260.515425
3,98004,386189.269358,304174.184435
4,98005,170193.881177,135050.680733
...,...,...,...
65,98177,194569.824278,385413.120756
66,98178,138042.126623,65083.421365
67,98188,56769.819873,44646.120830
68,98198,100124.916416,105570.846506


In [43]:
from joblib import dump, load
# Create the condensed model file

i = 0
while (i < len(list_of_models)):
   model = list_of_models[i]
   zipcode = unique_zipcodes[i]
   dump(model, "backend_root/projectApp/ml_models/" + str(zipcode))
   i += 1

In [100]:
import numpy as np
# Load the condensed model file
mdl = load('basic_model.joblib')

# Assume user input
userInput = {
   'bedrooms': 3,
   'bathrooms': 2,
   'sqft_living': 2200,
   'sqft_lot': 4000,
   'floors': 2
}

# Convert to pandas dataframe
df = pd.DataFrame([userInput])

# Predict based on input
y_pred = mdl.predict(df)

# Display the results
print(y_pred)


[602008.86649883]
